In [1]:
import pandas as pd
import os
import json
import numpy as np
from types import NoneType
import re
from IPython.display import Markdown, display
import ast
# Cho phép hiển thị toàn bộ nội dung transcript
# pd.set_option('display.max_colwidth', None)

os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

# Load json files to df


In [2]:
# @title Concat json files to df
# # Đường dẫn folder chứa JSON
# json_folder = 'data/content-features-top50WeeklyVideos'

# # Danh sách lưu kết quả
# json_data_list = []

# # Lặp qua tất cả các file JSON trong thư mục
# for filename in os.listdir(json_folder):
#     if not filename.endswith('.json'):
#         continue

#     file_path = os.path.join(json_folder, filename)
#     # loại bỏ phần .json để lấy video_id
#     video_id = os.path.splitext(filename)[0]

#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             content = f.read()
#         # loại bỏ ký tự điều khiển
#         content = re.sub(r'[\x00-\x1F\x7F]', '', content)
#         content = json.loads(content)
#         content['video.id'] = video_id  # thêm video.id vào để merge
#         json_data_list.append(content)
#     except Exception as e:
#         # print(f"❌ Lỗi khi đọc file {file_path}: {e}")
#         continue

# # Chuyển thành DataFrame từ danh sách JSON
# json_df = pd.DataFrame(json_data_list)
# json_df.info()

In [3]:
# json_df.to_parquet(
#     "data/content-features-dataset/content_features-raw-top50WeeklyVideos.parquet", index=False)

In [4]:
raw_content_df = pd.read_parquet(
    'data/content-features-dataset/content_features-raw-top50WeeklyVideos.parquet')
raw_content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   categories         3005 non-null   object
 1   audience_target    2806 non-null   object
 2   body_analysis      2811 non-null   object
 3   content_style      2809 non-null   object
 4   cta_content        2809 non-null   object
 5   cta_type           2809 non-null   object
 6   emotion_keywords   2810 non-null   object
 7   has_cta            2811 non-null   object
 8   highlight_phrases  2811 non-null   object
 9   hook_content       2811 non-null   object
 10  hook_type          2810 non-null   object
 11  pacing             2809 non-null   object
 12  structure_style    2809 non-null   object
 13  summarization      2812 non-null   object
 14  tone_of_voice      2812 non-null   object
 15  video.id           3005 non-null   object
dtypes: object(16)
memory usage: 375.8+ KB


Xử lý None value


In [5]:
# features_df['pacing'].astype('str').unique()
str_cols = []
for col in raw_content_df.columns:
    print(f"{col}: {raw_content_df[col].apply(type).unique()}")

    str_flag = raw_content_df[col].apply(lambda x: isinstance(x, str)).any()
    if str_flag:
        str_cols.append(col)

str_cols

categories: [<class 'numpy.ndarray'>]
audience_target: [<class 'numpy.ndarray'> <class 'NoneType'>]
body_analysis: [<class 'numpy.ndarray'> <class 'NoneType'>]
content_style: [<class 'numpy.ndarray'> <class 'NoneType'>]
cta_content: [<class 'numpy.ndarray'> <class 'NoneType'>]
cta_type: [<class 'numpy.ndarray'> <class 'NoneType'>]
emotion_keywords: [<class 'numpy.ndarray'> <class 'NoneType'>]
has_cta: [<class 'str'> <class 'NoneType'>]
highlight_phrases: [<class 'numpy.ndarray'> <class 'NoneType'>]
hook_content: [<class 'str'> <class 'NoneType'>]
hook_type: [<class 'numpy.ndarray'> <class 'NoneType'>]
pacing: [<class 'numpy.ndarray'> <class 'NoneType'>]
structure_style: [<class 'numpy.ndarray'> <class 'NoneType'>]
summarization: [<class 'str'> <class 'NoneType'>]
tone_of_voice: [<class 'numpy.ndarray'> <class 'NoneType'>]
video.id: [<class 'str'>]


['has_cta', 'hook_content', 'summarization', 'video.id']

In [6]:
content_df = raw_content_df.copy()

In [7]:
content_df['has_cta'] = raw_content_df['has_cta'].str.lower().map(
    {'true': True, 'false': False})
content_df['has_cta'] = raw_content_df['has_cta'].fillna(False).astype('bool')

In [8]:
def parse_list(x):
    if isinstance(x, np.ndarray):
        return list(x)
    if isinstance(x, NoneType):
        return []


def parse_string(x):
    if isinstance(x, str):
        return x
    if isinstance(x, NoneType):
        return ""


list_cols = set(raw_content_df.columns)-set(str_cols)-set(['video.id'])
str_cols = set(str_cols)-set(['has_cta'])

for col in raw_content_df.columns:
    if col in list_cols:
        content_df[col] = raw_content_df[col].apply(parse_list)
    elif col in str_cols:
        content_df[col] = raw_content_df[col].apply(parse_string)

In [9]:
# features_df['pacing'].astype('str').unique()
str_cols = []
for col in content_df.columns:
    print(f"{col}: {content_df[col].apply(type).unique()}")

    str_flag = content_df[col].apply(lambda x: isinstance(x, str)).any()
    if str_flag:
        str_cols.append(col)

str_cols

categories: [<class 'list'>]
audience_target: [<class 'list'>]
body_analysis: [<class 'list'>]
content_style: [<class 'list'>]
cta_content: [<class 'list'>]
cta_type: [<class 'list'>]
emotion_keywords: [<class 'list'>]
has_cta: [<class 'bool'>]
highlight_phrases: [<class 'list'>]
hook_content: [<class 'str'>]
hook_type: [<class 'list'>]
pacing: [<class 'list'>]
structure_style: [<class 'list'>]
summarization: [<class 'str'>]
tone_of_voice: [<class 'list'>]
video.id: [<class 'str'>]


['hook_content', 'summarization', 'video.id']

In [10]:
raw_transcript_df = pd.read_parquet(
    "data/transcripts-dataset/top_50_weekly_video_transcripts.parquet")
raw_transcript_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transcript          2940 non-null   object 
 1   takeaways           2992 non-null   object 
 2   has_call_to_action  3005 non-null   bool   
 3   has_curiosity_gap   3005 non-null   bool   
 4   video.id            3005 non-null   object 
 5   year                3005 non-null   int64  
 6   week                3005 non-null   int64  
 7   weekly_score_rank   3005 non-null   float64
 8   desc                3004 non-null   object 
dtypes: bool(2), float64(1), int64(2), object(4)
memory usage: 170.3+ KB


In [11]:
transcript_df = raw_transcript_df.copy()
transcript_df[['desc', 'transcript']] = transcript_df[[
    'desc', 'transcript']].fillna("")
transcript_df['takeaways'] = transcript_df['takeaways'].apply(parse_list)

In [12]:
transcript_df_to_merge = transcript_df[[
    'video.id', 'desc', 'transcript', 'takeaways', 'weekly_score_rank']]

In [13]:
full_df = pd.read_parquet('data/preprocessed_videos.parquet')
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70996 entries, 0 to 70995
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype                           
---  ------                           --------------  -----                           
 0   CategoryType                     70996 non-null  object                          
 1   author.downloadSetting           70996 non-null  object                          
 2   author.duetSetting               70996 non-null  object                          
 3   author.id                        70996 non-null  object                          
 4   author.nickname                  70996 non-null  object                          
 5   author.openFavorite              70996 non-null  bool                            
 6   author.secUid                    70996 non-null  object                          
 7   author.signature                 70996 non-null  object                          
 8   author.stitchSet

In [14]:
renamed_cols = {
    # 'author.uniqueId': 'username',
    'video.id': 'video.id',
    # "id": "id",
    "isAd": "isAd",
    "video.duration": "duration",
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    # "statsV2.playCount_rank": "views_rank",  # Lượt xem video
    # "statsV2.diggCount_rank": "likes_rank",  # Lượt thích video
    # "statsV2.commentCount_rank": "comments_rank",  # Lượt bình luận video
    # "statsV2.shareCount_rank": "shares_rank",  # Lượt chia sẻ video
    # 'statsV2.collectCount_rank': "collects_rank",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
    'createTime': 'createTime',
    'hashtags': 'hashtags',
    'hashtag_count': 'hashtag_count'
}

In [15]:
full_df_to_merge = full_df[renamed_cols.keys()]

In [16]:
merge_df = transcript_df_to_merge.merge(content_df, on='video.id', how='left').merge(
    full_df_to_merge, on='video.id', how='left')

In [17]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   video.id                    3005 non-null   object                          
 1   desc                        3005 non-null   object                          
 2   transcript                  3005 non-null   object                          
 3   takeaways                   3005 non-null   object                          
 4   weekly_score_rank           3005 non-null   float64                         
 5   categories                  3005 non-null   object                          
 6   audience_target             3005 non-null   object                          
 7   body_analysis               3005 non-null   object                          
 8   content_style               3005 non-null   object                  

In [18]:
merge_df.to_parquet(
    "data/content-features-dataset/content_features-with-stats-top50WeeklyVideos.parquet", index=False)